In [3]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from sklearn.cluster import DBSCAN, HDBSCAN
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.stats import kendalltau, norm
import cvxpy as cp
import calendar
from datetime import date
import pickle

from sb3_contrib import RecurrentPPO
import gymnasium as gym
from gymnasium import spaces
# from sklearn.linear_model import LinearRegression

import networkx as nx
from sklearn.cluster import SpectralClustering
from pandas.tseries.offsets import MonthEnd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [5]:
!pip install sb3_contrib gymnasium

In [6]:
folder_path = '/content/drive/My Drive/fednlp/ai/'
prices_info = pd.read_csv(f'{folder_path}prices_info.csv.gz', compression='gzip')
ratios_info = pd.read_csv(f'{folder_path}ratios.csv.gz', compression='gzip')
sector_info = pd.read_csv(f'{folder_path}sector_info.csv.gz', compression='gzip')
sp500 = pd.read_pickle(f'{folder_path}sp500.pkl')
relations = pd.read_csv(f'{folder_path}lorlxamlga34g2og.csv')
rels2 = pd.read_feather(f'{folder_path}econ_link_filt2.feather')

ratios = ratios_info.drop(columns=['gvkey','cusip','adate','qdate'])

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [10]:
tech_features = pd.read_feather(f"{folder_path}tech_features.feather")
trading_window = pd.read_feather(f"{folder_path}trading_window.feather")
lookback_correlation_pairs = pd.read_feather(f"{folder_path}lookback_correlation_pairs.feather")
lookback_resid = pd.read_feather(f"{folder_path}lookback_resid.feather")
forward_resid = pd.read_feather(f"{folder_path}forward_resid.feather")

# with open(f"{folder_path}all_ticker_univ.pkl", "rb") as f:
#   all_ticker_univ = pickle.load(f)

with open(f"{folder_path}universe_dict3.pkl", "rb") as f:
  universe_dict3 = pickle.load(f)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [11]:
rets = pd.read_feather(f"{folder_path}rets_cluster.feather")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [12]:
rets

,TICKER,DATETIME,RET,YEAR,MONTH,gsector
0,ORCL,2013-01-02,0.041116,2013,1,9
1,MSFT,2013-01-02,0.034081,2013,1,9
2,TROW,2013-01-02,0.028609,2013,1,3
3,HON,2013-01-02,0.020167,2013,1,9
4,LLTC,2013-01-02,0.049271,2013,1,9
...,...,...,...,...,...,...
33085192,VLUE,2024-12-24,0.0,2024,12,10
33096101,VLUE,2024-12-26,0.010373,2024,12,10
33107025,VLUE,2024-12-27,-0.00574,2024,12,10
33117966,VLUE,2024-12-30,-0.011593,2024,12,10


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [14]:
lookback_resid.tail()

,TICKER,gsector,LOOKBACK_START_DATE,LOOKBACK_END_DATE,DATETIME,YEAR,MONTH,RESID
5652746,FAST,0.0,2020-09-01,2021-08-31,2021-08-25,2021.0,8.0,0.009220
5652747,FAST,0.0,2020-09-01,2021-08-31,2021-08-26,2021.0,8.0,0.008924
5652748,FAST,0.0,2020-09-01,2021-08-31,2021-08-27,2021.0,8.0,-0.006069
5652749,FAST,0.0,2020-09-01,2021-08-31,2021-08-30,2021.0,8.0,0.002112
5652750,FAST,0.0,2020-09-01,2021-08-31,2021-08-31,2021.0,8.0,-0.003063


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [15]:
lookback_correlation_pairs.tail()

,YEAR,MONTH,TICKER_1,TICKER_2,SPEARMAN_RHO
1293816,2021,9,ARE,UNH,0.090909
1293817,2021,9,ARE,DG,-0.311124
1293818,2021,9,CNC,UNH,0.417278
1293819,2021,9,CNC,DG,-0.719932
1293820,2021,9,UNH,DG,-0.164314


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [18]:
import pandas as pd
import numpy as np

def generate_pairs_features(t1_close, t2_close, ticker1, ticker2, lookback_resid, window=30):
    """
    Helper function to calculate Z-Score, Half-Life, Hedge Ratio, and Residual Spreads.
    """
    # 1. Setup Price Data
    # Log prices are generally more stable for hedge ratios
    p1 = np.log(t1_close)
    p2 = np.log(t2_close)

    # 2. Integrate ETF Residuals
    # FIX: Handle duplicates in lookback_resid (caused by overlapping windows)

    # Process T1
    res1 = lookback_resid[lookback_resid['TICKER'] == ticker1].copy()
    res1['DATETIME'] = pd.to_datetime(res1['DATETIME'])
    # Drop duplicates on Date, keeping the most recent calculation
    res1 = res1.drop_duplicates(subset=['DATETIME'], keep='last')
    res1 = res1.set_index('DATETIME')['RESID'].rename('resid_t1')

    # Process T2
    res2 = lookback_resid[lookback_resid['TICKER'] == ticker2].copy()
    res2['DATETIME'] = pd.to_datetime(res2['DATETIME'])
    res2 = res2.drop_duplicates(subset=['DATETIME'], keep='last')
    res2 = res2.set_index('DATETIME')['RESID'].rename('resid_t2')

    # Align residuals with prices
    # We use the index from t1_close. We must ensure t1_close index is unique too (handled in main func)
    features = pd.DataFrame(index=t1_close.index)

    # Use reindex to strictly match the pricing dates (filling missing residuals with 0)
    features['resid_t1'] = res1.reindex(features.index).ffill().fillna(0)
    features['resid_t2'] = res2.reindex(features.index).ffill().fillna(0)

    # Feature: Residual Spread
    features['resid_spread'] = features['resid_t1'] - features['resid_t2']

    # 3. Rolling Hedge Ratio (Beta)
    r1 = p1.diff()
    r2 = p2.diff()

    roll_cov = r1.rolling(window=window).cov(r2)
    roll_var = r2.rolling(window=window).var()
    features['hedge_ratio'] = (roll_cov / roll_var).fillna(1.0)

    # 4. Spread & Z-Score
    features['spread'] = p1 - features['hedge_ratio'] * p2

    spread_mean = features['spread'].rolling(window=window).mean()
    spread_std = features['spread'].rolling(window=window).std()

    features['zscore'] = (features['spread'] - spread_mean) / (spread_std + 1e-8)

    # Feature: Delta
    features['delta'] = features['spread'].diff().fillna(0)

    # 5. Half-Life
    spread_lag = features['spread'].shift(1)
    rho = features['spread'].rolling(window=window).corr(spread_lag).clip(0.001, 0.999)
    features['half_life'] = -np.log(2) / np.log(rho)
    features['half_life'] = features['half_life'].fillna(window/2)

    return features


def prepare_pair_data_with_advanced_features(ticker1, ticker2, tech_features, rets, lookback_df, lookback_resid):
    """
    Master data preparation function with Robust Deduplication.
    """
    # -------------------------------------------------------
    # 1. FILTER & INDEX RAW INPUTS (With Deduplication)
    # -------------------------------------------------------
    drop_cols = ['date', 'ticker', 'close', 'open', 'high', 'low', 'volume',
                 'dlyretx', 'dlybid', 'dlyask', 'RET', 'RET_LAG_1', 'RET_LAG_2', 'RET_LAG_3', 'RET_LAG_4']

    # T1 Data
    df1 = tech_features[tech_features['ticker'] == ticker1].copy()
    df1['date'] = pd.to_datetime(df1['date'])
    # FIX: Ensure unique index for prices
    df1 = df1.drop_duplicates(subset=['date'], keep='last')
    df1 = df1.set_index('date').sort_index()
    t1_close = df1['close']

    # T2 Data
    df2 = tech_features[tech_features['ticker'] == ticker2].copy()
    df2['date'] = pd.to_datetime(df2['date'])
    # FIX: Ensure unique index for prices
    df2 = df2.drop_duplicates(subset=['date'], keep='last')
    df2 = df2.set_index('date').sort_index()
    t2_close = df2['close']

    # Raw Returns (For PnL)
    r1 = rets[rets['TICKER'] == ticker1].copy()
    r1['DATETIME'] = pd.to_datetime(r1['DATETIME'])
    # FIX: Ensure unique index for returns
    r1 = r1.drop_duplicates(subset=['DATETIME'], keep='last')
    r1 = r1.set_index('DATETIME')['RET'].sort_index().rename('raw_ret')

    r2 = rets[rets['TICKER'] == ticker2].copy()
    r2['DATETIME'] = pd.to_datetime(r2['DATETIME'])
    # FIX: Ensure unique index for returns
    r2 = r2.drop_duplicates(subset=['DATETIME'], keep='last')
    r2 = r2.set_index('DATETIME')['RET'].sort_index().rename('raw_ret')

    # -------------------------------------------------------
    # 2. GENERATE ADVANCED PAIRS FEATURES
    # -------------------------------------------------------
    # Now that indices are guaranteed unique, this will run without error
    pairs_feats = generate_pairs_features(t1_close, t2_close, ticker1, ticker2, lookback_resid)

    # -------------------------------------------------------
    # 3. ALIGN EVERYTHING
    # -------------------------------------------------------
    df1_clean = df1.drop(columns=[c for c in drop_cols if c in df1.columns], errors='ignore').select_dtypes(include=[np.number]).add_prefix('t1_')
    df2_clean = df2.drop(columns=[c for c in drop_cols if c in df2.columns], errors='ignore').select_dtypes(include=[np.number]).add_prefix('t2_')

    aligned = pd.concat([
        df1_clean,
        df2_clean,
        pairs_feats,
        r1.rename('ret1'),
        r2.rename('ret2')
    ], axis=1, join='inner')

    # -------------------------------------------------------
    # 4. ADD CORRELATIONS
    # -------------------------------------------------------
    # A. Rolling Pearson
    aligned['rolling_pearson'] = aligned['ret1'].rolling(window=30).corr(aligned['ret2']).fillna(0)

    # B. Monthly Spearman
    mask_pair = (
        ((lookback_df['TICKER_1'] == ticker1) & (lookback_df['TICKER_2'] == ticker2)) |
        ((lookback_df['TICKER_1'] == ticker2) & (lookback_df['TICKER_2'] == ticker1))
    )
    pair_spearman = lookback_df[mask_pair].copy()

    # Deduplicate Spearman data just in case
    pair_spearman = pair_spearman.drop_duplicates(subset=['YEAR', 'MONTH'])

    daily_dates = pd.DataFrame(index=aligned.index)
    daily_dates['YEAR'] = daily_dates.index.year
    daily_dates['MONTH'] = daily_dates.index.month

    merged_spearman = pd.merge(
        daily_dates.reset_index(),
        pair_spearman[['YEAR', 'MONTH', 'SPEARMAN_RHO']],
        on=['YEAR', 'MONTH'],
        how='left'
    ).drop_duplicates(subset=['index']).set_index('index')

    aligned['monthly_spearman'] = merged_spearman['SPEARMAN_RHO'].reindex(aligned.index).fillna(0)

    # -------------------------------------------------------
    # 5. FINAL CLEANUP
    # -------------------------------------------------------
    aligned = aligned.dropna()

    final_obs = aligned.drop(columns=['ret1', 'ret2'])
    final_ret1 = aligned['ret1']
    final_ret2 = aligned['ret2']

    return final_obs, final_ret1, final_ret2

In [20]:
?prepare_pair_data_with_advanced_features

In [21]:
def select_monthly_dynamic_portfolio(lookback_corr, rets, top_n_per_sector=2):
    """
    Selects top N pairs per cluster (gsector) for each month based on Correlation strength.
    """
    print("Selecting monthly portfolios...")

    # 1. Map Tickers to Sectors (gsector)
    # We create a lookup table from the 'rets' dataframe
    # We drop duplicates to get a unique Ticker -> Sector mapping
    # (Assuming tickers don't change sectors frequently, we take the most recent)
    ticker_sector_map = rets[['TICKER', 'gsector']].drop_duplicates(subset=['TICKER'], keep='last').set_index('TICKER')['gsector'].to_dict()

    # 2. Helper to identify pair sector
    def get_pair_sector(row):
        s1 = ticker_sector_map.get(row['TICKER_1'])
        s2 = ticker_sector_map.get(row['TICKER_2'])
        # We only trade pairs within the same sector (Cluster)
        if s1 == s2 and s1 is not None:
            return s1
        return -1 # Invalid or Cross-sector

    # 3. Enhance Correlation Table
    df = lookback_corr.copy()
    df['gsector'] = df.apply(get_pair_sector, axis=1)

    # Filter valid within-cluster pairs
    df = df[df['gsector'] != -1]

    # Create Absolute Correlation for ranking (High neg correlation is also good)
    df['abs_rho'] = df['SPEARMAN_RHO'].abs()

    # 4. Group by Year/Month/Sector and pick Top N
    # Sort by Year, Month, Sector, and Strength
    df = df.sort_values(['YEAR', 'MONTH', 'gsector', 'abs_rho'], ascending=[True, True, True, False])

    # Select top N
    selected_portfolio = df.groupby(['YEAR', 'MONTH', 'gsector']).head(top_n_per_sector)

    print(f"Total Portfolio Universes generated: {len(selected_portfolio.groupby(['YEAR', 'MONTH']))} months")
    print(f"Total Unique Pairs to Train: {len(selected_portfolio[['TICKER_1', 'TICKER_2']].drop_duplicates())}")

    return selected_portfolio


import os
import matplotlib.pyplot as plt
from sb3_contrib import RecurrentPPO

def backtest_dynamic_portfolio(
    monthly_portfolio,
    tech_features,
    rets,
    lookback_resid,
    lookback_corr,
    model_folder="models",
    initial_capital=100_000,
    capital_allocation_per_pair=10_000
):
    """
    Simulates a portfolio where constituents change every month.
    """
    # 1. Identify Time Horizon
    # Get all unique Year-Months in chronological order
    timesteps = monthly_portfolio[['YEAR', 'MONTH']].drop_duplicates().sort_values(['YEAR', 'MONTH'])

    # Global Portfolio Tracker
    # Index will be dates, Columns will be 'Total_Equity'
    portfolio_equity = pd.Series(dtype=float)
    current_cash = initial_capital

    # To track individual pair performance for debugging
    active_positions_log = []

    print(f"Starting Dynamic Backtest over {len(timesteps)} months...")

    # ====================================================
    # MONTHLY LOOP
    # ====================================================
    for _, time_row in timesteps.iterrows():
        yr, mo = int(time_row['YEAR']), int(time_row['MONTH'])

        # A. Determine Start/End dates for this month
        # We find the specific trading days available in 'tech_features' for this month
        month_mask = (tech_features['date'].dt.year == yr) & (tech_features['date'].dt.month == mo)
        month_dates = tech_features.loc[month_mask, 'date'].unique()

        if len(month_dates) == 0:
            continue

        month_dates = np.sort(month_dates)
        start_date, end_date = month_dates[0], month_dates[-1]

        print(f"Processing {yr}-{mo} ({len(month_dates)} trading days)...")

        # B. Get Active Pairs for this Month
        active_pairs_df = monthly_portfolio[
            (monthly_portfolio['YEAR'] == yr) &
            (monthly_portfolio['MONTH'] == mo)
        ]

        # Dictionary to store daily PnL for this month's active pairs
        # Key: Date, Value: Sum of PnL
        monthly_pnl_agg = {d: 0.0 for d in month_dates}

        # ====================================================
        # PAIR LOOP (Run Inference for active pairs)
        # ====================================================
        for _, pair_row in active_pairs_df.iterrows():
            t1, t2 = pair_row['TICKER_1'], pair_row['TICKER_2']
            model_name = f"{model_folder}/ppo_{t1}_{t2}.zip"

            # Skip if model wasn't trained (or train on the fly if you prefer)
            if not os.path.exists(model_name):
                # print(f"Warning: Model for {t1}-{t2} not found. Skipping.")
                continue

            try:
                # 1. Prepare Data for THIS PAIR
                # We get the full history, then slice for the current month
                obs, ret1, ret2, _, _ = prepare_pair_data_with_advanced_features(
                    t1, t2, tech_features, rets, lookback_correlation_pairs, lookback_resid
                    # Note: You can add lookback_resid here if you integrated the advanced features function
                )

                # 2. Slice Data for Current Month Only
                # We need to map the integer index of 'obs' to actual dates to slice
                # prepare_pair_data returns dataframes with Date index (if using my previous code)
                # If obs is numpy/reset_index, we need the original index.
                # Let's assume prepare_pair_data output kept the DatetimeIndex on 'ret1'/'ret2'

                # Align using the return index (which has dates)
                mask = (ret1.index >= start_date) & (ret1.index <= end_date)

                if mask.sum() == 0: continue

                # Extract chunk
                obs_chunk = obs.loc[mask]
                ret1_chunk = ret1.loc[mask]
                ret2_chunk = ret2.loc[mask]

                # 3. Load Model
                model = RecurrentPPO.load(model_name, device="gpu")

                # 4. Run Inference
                # We instantiate a temporary env just for calculation logic
                # Initial balance doesn't matter much here, we track PnL relative to allocation
                env = PairsTradingEnv(
                    obs_chunk, obs_chunk, # Dummy dup for df2 if structure requires it
                    ret1_chunk, ret2_chunk,
                    pd.DataFrame() # dummy corr
                )

                # Manually Override Env Data with correct slice
                env.obs1 = obs_chunk # Assuming your env handles DF, or convert to values
                # If your Env expects pre-processed arrays:
                # env.obs1 = obs_chunk.reset_index(drop=True).astype(np.float32)
                # ... update other env vars ...
                # Easier way: Use the Step Logic directly without full Env overhead for speed

                # MANUAL INFERENCE LOOP (Faster/Cleaner for backtest)
                lstm_states = None
                episode_start = np.ones((1,), dtype=bool)
                pos = 0.0

                for i in range(len(obs_chunk)):
                    # Construct Observation Vector manually
                    # (Ensure this matches your Env's _next_observation logic exactly)
                    o1 = obs_chunk.iloc[i].values.astype(np.float32)
                    # For this snippet, assuming obs_chunk contains all needed features
                    # If your env expects separate dfs, you need to handle that.
                    # Let's assume obs_chunk is the FUSED dataframe returned by advanced_prep

                    # Observation: Features + Position
                    step_obs = np.concatenate([o1, [pos]])

                    # Predict
                    action, lstm_states = model.predict(step_obs, state=lstm_states, episode_start=episode_start, deterministic=True)
                    target_pos = float(action[0])

                    # Calculate PnL (Next Day Return)
                    # We can't calc PnL for the very last day of the chunk if we don't have tomorrow's price
                    if i < len(obs_chunk) - 1:
                        r1_val = ret1_chunk.iloc[i+1]
                        r2_val = ret2_chunk.iloc[i+1]

                        gross = target_pos * (r1_val - r2_val)
                        cost = abs(target_pos - pos) * 0.0005
                        net_pnl_pct = gross - cost

                        # Add Dollar PnL to Daily Aggregator
                        date_key = ret1_chunk.index[i]
                        monthly_pnl_agg[date_key] += (net_pnl_pct * capital_allocation_per_pair)

                    pos = target_pos
                    episode_start = np.zeros((1,), dtype=bool)

            except Exception as e:
                # print(f"Error on {t1}-{t2}: {e}")
                pass

        # C. Update Global Portfolio Equity
        # Add the daily PnL sums to our running cash balance
        sorted_dates = sorted(monthly_pnl_agg.keys())
        for d in sorted_dates:
            daily_profit = monthly_pnl_agg[d]
            current_cash += daily_profit
            # Record Equity
            portfolio_equity[d] = current_cash

    return portfolio_equity

In [45]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd

class PairsTradingEnvOptimized(gym.Env):
    def __init__(self, fused_obs, ret1, ret2, initial_balance=10000):
        super().__init__()
        # Initialize with data (same as before)
        self.update_data(fused_obs, ret1, ret2, initial_balance)

        # Define Spaces (Fixed based on initial data shape)
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        # Obs dim = features + position (1)
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf,
            shape=(self.n_features + 1,),
            dtype=np.float32
        )

    def update_data(self, fused_obs, ret1, ret2, initial_balance=10000):
        """
        Hot-swaps the dataset without killing the process.
        """
        self.obs_matrix = fused_obs.reset_index(drop=True).values.astype(np.float32)
        self.r1_array = ret1.reset_index(drop=True).values.astype(np.float32)
        self.r2_array = ret2.reset_index(drop=True).values.astype(np.float32)

        # Update Dimensions
        self.n_steps = len(self.obs_matrix)
        self.n_features = self.obs_matrix.shape[1]

        # Reset State
        self.current_step = 0
        self.balance = initial_balance
        self.positions = 0.0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 0
        self.balance = 10000.0
        self.positions = 0.0
        return self._next_observation(), {}

    def _next_observation(self):
        # Fast numpy indexing
        features = self.obs_matrix[self.current_step]
        position = np.array([self.positions], dtype=np.float32)

        # Concatenate features with current position state
        return np.concatenate([features, position])

    def step(self, action):
        target_position = float(action[0])

        # Check boundary
        if self.current_step >= self.n_steps - 1:
            return self._next_observation(), 0.0, True, False, {}

        # Calculate Returns (using T+1 raw returns)
        r1 = self.r1_array[self.current_step + 1]
        r2 = self.r2_array[self.current_step + 1]

        # PnL: Position * (Spread Return)
        gross_return = target_position * (r1 - r2)

        # Transaction Costs (5bps per turnover unit)
        turnover = abs(target_position - self.positions)
        cost = turnover * 0.0005

        net_return = gross_return - cost

        # Update State
        self.balance *= (1 + net_return)
        self.positions = target_position
        self.current_step += 1

        # Termination conditions
        terminated = self.current_step >= self.n_steps - 1

        # Stop Loss: Kill episode if 50% drawdown
        if self.balance < 5000:
            terminated = True
            net_return = -1.0 # Significant Penalty

        return self._next_observation(), net_return, terminated, False, {}

In [23]:
import pandas as pd

def get_target_universe(lookback_corr, rets, top_n_per_sector=5):
    """
    Selects the top N correlated pairs from each sector, ensuring NO self-pairs.
    """
    print("Generating Target Universe...")

    # 1. Map Tickers to Sectors
    ticker_map = rets[['TICKER', 'gsector']].drop_duplicates(subset=['TICKER'], keep='last').set_index('TICKER')['gsector'].to_dict()

    def get_sector(row):
        t1, t2 = row['TICKER_1'], row['TICKER_2']

        # CRITICAL FIX: Reject if Ticker 1 == Ticker 2
        if t1 == t2:
            return -1

        s1 = ticker_map.get(t1)
        s2 = ticker_map.get(t2)

        # Only keep pairs in the same cluster
        return s1 if s1 == s2 and s1 is not None else -1

    # 2. Filter and Rank
    df = lookback_corr.copy()
    df['gsector'] = df.apply(get_sector, axis=1)

    # Remove cross-sector pairs AND self-pairs (marked as -1 above)
    df = df[df['gsector'] != -1]

    # Sort by strength of correlation
    df['abs_rho'] = df['SPEARMAN_RHO'].abs()

    # Remove Duplicate Combinations (e.g. A-B and B-A)
    # We normalize by sorting tickers alphabetically per row
    # This creates a 'signature' so A-B and B-A become the same string
    df['pair_key'] = df.apply(lambda r: "-".join(sorted([r['TICKER_1'], r['TICKER_2']])), axis=1)
    df = df.drop_duplicates(subset=['pair_key'])

    # Pick Top N per Sector
    target_universe = df.sort_values('abs_rho', ascending=False).groupby('gsector').head(top_n_per_sector)

    print(f"Selected {len(target_universe)} unique pairs across {target_universe['gsector'].nunique()} sectors.")
    return target_universe[['TICKER_1', 'TICKER_2', 'gsector', 'SPEARMAN_RHO']]

In [30]:
lookback_correlation_pairs = lookback_correlation_pairs[lookback_correlation_pairs['TICKER_1'] != lookback_correlation_pairs['TICKER_2'] ]

In [37]:
lookback_correlation_pairs = lookback_correlation_pairs[lookback_correlation_pairs['SPEARMAN_RHO'] < 0.98]

In [38]:
lookback_correlation_pairs['SPEARMAN_RHO'].max()

0.9799440155033992

In [49]:
import os
import torch
import gc
import numpy as np
import pandas as pd
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

def train_mass_universe_persistent(
    universe_df,
    tech_features,
    rets,
    lookback_corr,
    lookback_resid,
    save_folder="models_v1"
):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Starting Persistent Factory Training on {device}...")

    # -----------------------------------------------------
    # 1. INITIALIZE VEC_ENV (ONCE)
    # -----------------------------------------------------
    # We need to run data prep ONCE to get the correct Feature Shape
    # to initialize the environment workers correctly.
    first_row = universe_df.iloc[0]
    t1_init, t2_init = first_row['TICKER_1'], first_row['TICKER_2']

    obs_init, r1_init, r2_init = prepare_pair_data_with_advanced_features(
        t1_init, t2_init, tech_features, rets, lookback_corr, lookback_resid
    )

    # Define the factory for the persistent workers
    def make_env():
        return PairsTradingEnvOptimized(
            obs_init.copy(), r1_init.copy(), r2_init.copy()
        )

    # Spawn processes ONCE (High overhead happens here only)
    print("Spawning worker processes (this takes a few seconds)...")
    vec_env = make_vec_env(
        make_env,
        n_envs=32, # High parallelism
        vec_env_cls=SubprocVecEnv,
        vec_env_kwargs={"start_method": "spawn"}
    )

    print(f'vec_env loaded ')

    feature_shape = obs_init.shape[1]
    print(f"Workers ready. Expecting {feature_shape} features.")

    # -----------------------------------------------------
    # 2. FAST LOOP
    # -----------------------------------------------------
    total_pairs = len(universe_df)

    for i, (_, row) in enumerate(universe_df.iterrows()):
        t1, t2 = row['TICKER_1'], row['TICKER_2']
        model_name = f"{save_folder}/ppo_{t1}_{t2}"

        if os.path.exists(f"{model_name}.zip"):
            print(f"[{i+1}/{total_pairs}] Skipping {t1}-{t2}...")
            continue

        print(f"[{i+1}/{total_pairs}] Training {t1} vs {t2}...")

        try:
            # A. Prepare New Data
            final_obs, final_ret1, final_ret2 = prepare_pair_data_with_advanced_features(
                t1, t2, tech_features, rets, lookback_corr, lookback_resid
            )

            # Validation
            if len(final_obs) < 252:
                print("  -> Skipped: Insufficient data")
                continue

            if final_obs.shape[1] != feature_shape:
                print(f"  -> Skipped: Feature mismatch ({final_obs.shape[1]} vs {feature_shape})")
                continue

            # B. INJECT DATA into existing workers
            # env_method calls 'update_data' on all 32 subprocesses
            # This pickles the new data and sends it over the pipe (very fast)
            vec_env.env_method(
                "update_data",
                final_obs.copy(),
                final_ret1.copy(),
                final_ret2.copy()
            )

            # Force reset to apply new data limits (n_steps)
            vec_env.reset()

            print(f"device - {device}")
            # C. Train
            # We create a new PPO object, but attach it to the EXISTING vec_env
            model = RecurrentPPO(
                "MlpLstmPolicy",
                vec_env,
                verbose=0,
                device=device,
                learning_rate=3e-4,
                n_steps=512,
                batch_size=4096,
                n_epochs=10,
                gamma=0.99,
                ent_coef=0.01,
                policy_kwargs=dict(
                    enable_critic_lstm=False,
                    lstm_hidden_size=512,
                    net_arch=dict(pi=[512, 512], vf=[512, 512])
                )
            )

            model.learn(total_timesteps=150_000)
            model.save(model_name)

            # Cleanup only the model, KEEP the env
            del model
            torch.cuda.empty_cache()

            print("  -> Success.")

        except Exception as e:
            print(f"  -> Error: {e}")

    # Close Env only at the very end
    vec_env.close()
    print("All training complete.")

In [51]:
# 1. Generate Universe (e.g., top 3 pairs from each of the 10 clusters)
target_pairs = get_target_universe(lookback_correlation_pairs, rets, top_n_per_sector=3)

# 2. Run Training
# This will create a folder 'models_v1' and fill it with .zip files
train_mass_universe_persistent(
    target_pairs,
    tech_features,
    rets,
    lookback_correlation_pairs,
    lookback_resid,
    save_folder=f"{folder_path}models_v1"
)

Generating Target Universe...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Selected 30 pairs across 10 sectors.
Starting Persistent Factory Training on cuda...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Spawning worker processes (this takes a few seconds)...


...

  -> Success.
[29/30] Training KRFT vs BBBY...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

device - cuda
  -> Success.
[30/30] Training EQR vs AVB...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

device - cuda
  -> Success.
All training complete.


In [ ]:
# 1. Select the Dynamic Portfolio
# This tells us WHO to trade and WHEN
dynamic_portfolio_df = select_monthly_dynamic_portfolio(lookback_correlation_pairs, rets, top_n_per_sector=2)

# 2. Train Models (If needed)
# This loops through ALL unique pairs ever selected and ensures a model exists
unique_pairs = dynamic_portfolio_df[['TICKER_1', 'TICKER_2']].drop_duplicates()
print(f"Ensuring models exist for {len(unique_pairs)} unique pairs...")

# Reuse your training function (train_portfolio) passing unique_pairs
# train_portfolio(unique_pairs, tech_features, rets, lookback_resid, folder="models")

# 3. Run Dynamic Backtest
equity_curve = backtest_dynamic_portfolio(
    monthly_portfolio=dynamic_portfolio_df,
    tech_features=tech_features,
    rets=rets,
    lookback_resid=lookback_resid,
    lookback_corr=lookback_correlation_pairs,
    model_folder="models",
    initial_capital=100_000,
    capital_allocation_per_pair=10_000 # Allocating $10k per pair
)

# 4. Analyze Results
equity_curve.index = pd.to_datetime(equity_curve.index)
equity_curve = equity_curve.sort_index()

# Plot
plt.figure(figsize=(12, 6))
plt.plot(equity_curve)
plt.title("Dynamic Monthly Pairs Portfolio (Sector Clustered + RL)")
plt.ylabel("Portfolio Equity ($)")
plt.grid(True, alpha=0.3)
plt.show()

# Stats
total_ret = (equity_curve.iloc[-1] / equity_curve.iloc[0]) - 1
print(f"Total Return: {total_ret*100:.2f}%")
print(f"Final Equity: ${equity_curve.iloc[-1]:,.2f}")

In [ ]:
def analyze_results(results, ticker1, ticker2):
    """
    Calculates Sharpe, Max Drawdown, and plots performance.
    """
    # 1. Calculate Metrics
    results['Daily_Return'] = results['Portfolio_Value'].pct_change().fillna(0)

    # Cumulative Return
    total_ret = (results['Portfolio_Value'].iloc[-1] / results['Portfolio_Value'].iloc[0]) - 1

    # Sharpe Ratio (Assuming 252 trading days, risk-free=0)
    avg_ret = results['Daily_Return'].mean()
    std_ret = results['Daily_Return'].std()
    sharpe = (avg_ret / std_ret) * np.sqrt(252) if std_ret > 0 else 0

    # Max Drawdown
    cum_ret = (1 + results['Daily_Return']).cumprod()
    peak = cum_ret.cummax()
    drawdown = (cum_ret - peak) / peak
    max_drawdown = drawdown.min()

    print(f"--- Backtest Results: {ticker1} vs {ticker2} ---")
    print(f"Total Return:   {total_ret*100:.2f}%")
    print(f"Sharpe Ratio:   {sharpe:.2f}")
    print(f"Max Drawdown:   {max_drawdown*100:.2f}%")
    print(f"Final Balance:  ${results['Portfolio_Value'].iloc[-1]:,.2f}")

    # 2. Plotting
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 12), sharex=True)

    # Equity Curve
    ax1.plot(results.index, results['Portfolio_Value'], label='RL Strategy', color='green')
    ax1.set_title(f'Equity Curve: {ticker1}-{ticker2}')
    ax1.set_ylabel('Portfolio Value ($)')
    ax1.grid(True, alpha=0.3)
    ax1.legend()

    # Positions (Hedge Ratio / Weights)
    ax2.plot(results.index, results['Position'], label='Net Position', color='blue', linewidth=1)
    ax2.axhline(0, color='black', linestyle='--', alpha=0.5)
    ax2.set_title('Agent Positioning (-1 Short Spread, +1 Long Spread)')
    ax2.set_ylabel('Position')
    ax2.grid(True, alpha=0.3)

    # Drawdown
    ax3.fill_between(results.index, drawdown, 0, color='red', alpha=0.3)
    ax3.set_title('Drawdown')
    ax3.set_ylabel('% Drawdown')
    ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    return results

In [ ]:
import os
import matplotlib.pyplot as plt
from sb3_contrib import RecurrentPPO

def backtest_dynamic_portfolio(
    monthly_portfolio,
    tech_features,
    rets,
    lookback_resid,
    lookback_corr,
    model_folder="models",
    initial_capital=100_000,
    capital_allocation_per_pair=10_000
):
    """
    Backtests the dynamic portfolio and returns equity curve + position history.
    """
    # 1. Identify Time Horizon
    timesteps = monthly_portfolio[['YEAR', 'MONTH']].drop_duplicates().sort_values(['YEAR', 'MONTH'])

    # Global Trackers
    portfolio_equity = pd.Series(dtype=float)
    current_cash = initial_capital

    # Dictionary to store positions: { 'Ticker1-Ticker2': pd.Series(pos, index=dates) }
    # We will combine this into a DataFrame at the end
    all_positions_dict = {}

    print(f"Starting Dynamic Backtest over {len(timesteps)} months...")

    # ====================================================
    # MONTHLY LOOP
    # ====================================================
    for _, time_row in timesteps.iterrows():
        yr, mo = int(time_row['YEAR']), int(time_row['MONTH'])

        # A. Determine Trading Days
        month_mask = (tech_features['date'].dt.year == yr) & (tech_features['date'].dt.month == mo)
        month_dates = tech_features.loc[month_mask, 'date'].unique()

        if len(month_dates) == 0:
            continue

        month_dates = np.sort(month_dates)
        start_date, end_date = month_dates[0], month_dates[-1]

        print(f"Processing {yr}-{mo}...")

        # B. Get Active Pairs
        active_pairs_df = monthly_portfolio[
            (monthly_portfolio['YEAR'] == yr) &
            (monthly_portfolio['MONTH'] == mo)
        ]

        # Daily PnL Aggregator for this month
        monthly_pnl_agg = {d: 0.0 for d in month_dates}

        # ====================================================
        # PAIR LOOP
        # ====================================================
        for _, pair_row in active_pairs_df.iterrows():
            t1, t2 = pair_row['TICKER_1'], pair_row['TICKER_2']
            pair_key = f"{t1}-{t2}"
            model_name = f"{model_folder}/ppo_{t1}_{t2}.zip"

            if not os.path.exists(model_name):
                # print(f"Skipping {pair_key} (No Model)")
                continue

            try:
                # 1. Prepare Data
                obs, ret1, ret2 = prepare_pair_data_with_advanced_features(
                    t1, t2, tech_features, rets, lookback_corr, lookback_resid
                )

                # 2. Slice for Month
                mask = (ret1.index >= start_date) & (ret1.index <= end_date)
                if mask.sum() == 0: continue

                obs_chunk = obs.loc[mask]
                ret1_chunk = ret1.loc[mask]
                ret2_chunk = ret2.loc[mask]

                # 3. Load Model (CPU is faster for sequential loop)
                model = RecurrentPPO.load(model_name, device="cpu")

                # 4. Inference Loop
                lstm_states = None
                episode_start = np.ones((1,), dtype=bool)
                pos = 0.0

                daily_pos_list = []

                # Convert DataFrame to numpy for speed
                obs_values = obs_chunk.values.astype(np.float32)

                for i in range(len(obs_chunk)):
                    # Construct Observation: [Features + Position]
                    features = obs_values[i]
                    step_obs = np.concatenate([features, [pos]])

                    # Predict
                    action, lstm_states = model.predict(step_obs, state=lstm_states, episode_start=episode_start, deterministic=True)
                    target_pos = float(action[0])

                    # Store Position for this day
                    daily_pos_list.append(target_pos)

                    # Calculate PnL (applied to NEXT day's return)
                    if i < len(obs_chunk) - 1:
                        # Return of T+1
                        r1_val = ret1_chunk.iloc[i+1]
                        r2_val = ret2_chunk.iloc[i+1]

                        gross = target_pos * (r1_val - r2_val)
                        cost = abs(target_pos - pos) * 0.0005 # 5bps cost
                        net_pnl_pct = gross - cost

                        # Add to Monthly Aggregator
                        date_key = ret1_chunk.index[i] # PnL realized at end of day i (technically i+1 open/close, mapped to i for accounting)
                        monthly_pnl_agg[date_key] += (net_pnl_pct * capital_allocation_per_pair)

                    pos = target_pos
                    episode_start = np.zeros((1,), dtype=bool)

                # Store positions for this pair
                # Pad if lengths mismatch due to end-of-month logic
                pos_series = pd.Series(daily_pos_list, index=obs_chunk.index)
                if pair_key not in all_positions_dict:
                    all_positions_dict[pair_key] = pos_series
                else:
                    # Append if pair was traded in previous months
                    all_positions_dict[pair_key] = pd.concat([all_positions_dict[pair_key], pos_series])

            except Exception as e:
                # print(f"Error {t1}-{t2}: {e}")
                pass

        # C. Update Equity Curve
        sorted_dates = sorted(monthly_pnl_agg.keys())
        for d in sorted_dates:
            daily_profit = monthly_pnl_agg[d]
            current_cash += daily_profit
            portfolio_equity[d] = current_cash

    # Final Formatting
    portfolio_equity.index = pd.to_datetime(portfolio_equity.index)
    portfolio_equity = portfolio_equity.sort_index()

    # Combine positions into a wide DataFrame
    positions_df = pd.DataFrame(all_positions_dict).sort_index()

    return portfolio_equity, positions_df

In [ ]:
# 1. Select Portfolio
dynamic_portfolio_df = select_monthly_dynamic_portfolio(lookback_correlation_pairs, rets, top_n_per_sector=2)

# 2. Run Backtest
equity_curve, positions_df = backtest_dynamic_portfolio(
    monthly_portfolio=dynamic_portfolio_df,
    tech_features=tech_features,
    rets=rets,
    lookback_resid=lookback_resid,
    lookback_corr=lookback_correlation_pairs,
    model_folder=f"{folder_path}models_v1",
    initial_capital=100_000,
    capital_allocation_per_pair=10_000
)

# 3. Analyze
print(f"Final Equity: ${equity_curve.iloc[-1]:,.2f}")
plt.figure(figsize=(12, 6))
plt.plot(equity_curve, label="Portfolio Value")
plt.title("Dynamic Portfolio Equity")
plt.legend()
plt.show()

# Show positions sample
print(positions_df.tail())